# **Extract DayMET Data**

By Bridget Bittmann

Date created: March 28, 2022

Date editted: May 4, 2022

Purpose: This script extracts DayMET data from Google Earth Engine using the Earth Engine API in Python. This script also copied over a SSEBop ET data dataset and calculated zonal stats based on polygons. 

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=mgv9GunaiIGPyoeQC53nqe24gYixM9vFiDJjEw8IWdU&tc=ybUQYJNzG4T0-88mtiSEcrF5ly5o3E6KUThTV4zkCN0&cc=Z9b659Zemir5zK1KbJo6idUNonZhwkdLqPDdRF-lRP0

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi5Hj6w2gtSXFF0xiQOyqNDMv16zO7VjdXoEGW9G0R_IHdj633GXtg

Successfully saved authorization token.


In [2]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 12.9 MB/s 
     |████████████████████████████████| 16.7 MB 390 kB/s 
     |████████████████████████████████| 6.3 MB 39.4 MB/s 


In [3]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Go to correct folder in Google Drive
%cd drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
climate_stats/         irrig_lbrb/   masked/        seb_change.png
diversion_timeseries/  IrrMapper/    ny_change.png  set_change.png
fu_change.png          LBRB_shp/     output_files/  subset_test_shp/
irrigation_companies/  lcmap_files/  POUs/


In [ ]:
## ----------------------------------- ## 
## 1. Import shapefile to clip dataset ##
## ----------------------------------- ## 

shp_file = 'POUs/POUs_EDIT_051822_Merge.shp'
subset = emap.shp_to_ee(shp_file)

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(subset, 0, 2), {}, 'POU')
map.addLayerControl()
map

In [ ]:
## --------------------------------------- ##
## 2. IMPORT THE DAYMET DATA FOR MAX TEMPS ##
## --------------------------------------- ##

years = np.arange(1987,2021)
mean_max = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection
  mxtmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(years[i])}) #select the bands to analyze
  mean_max.append(mxtmp) #calculate the mean across all pixels

means_max_temp = ee.ImageCollection(mean_max) #convert list of image to image collection for zonal stats command

maximumTemperatureVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(means_max_temp, maximumTemperatureVis, 'tmax')
Map

In [ ]:
## Ask kendra if I should use this or an automated call for zonal stats, either works
## and both csvs need to be editted to an extent

## could define a spatial reduce function

def spatial_reduction(image_collection, shape_file, reducer, scale, file_name, format, output_folder, name):

  '''
  This function is used to produce zonal statistics out of an image collection in GEE.

  Variables:
  image_collection: A time series of ee.ImageCollection
  shapefile: The spatial area to reduce the image to. 
  reducer: The ee.Reducer to calculate. It can be a mixture of reducers or just one.
  scale: Pixel size
  file_name: The output table file name.
  format: The output table format, can include 'CSV', "GeoJSON", "KML", "KMZ", or "SHP", or "TFRecord".
  output_folder: The file path for the output file. 
  name: The climate statistic you are calculating (i.e., 'precip_mean')
  '''
  reduce_step = ee.FeatureCollection(image_collection.map(lambda image: image.reduceRegions(collection=shape_file,
                                                                                  scale=scale,
                                                                                  reducer=reducer)))
  flat_reduction = reduce_step.flatten()
  task = ee.batch.Export.table.toDrive(collection = flat_reduction,
                                     description=file_name,
                                     fileNamePrefix=file_name,
                                     fileFormat=format,
                                     folder=output_folder
  )
  task.start()
  while(True):
    if 'COMPLETED' not in task.status().values():
      print(task.status())
      time.sleep(10)
    else:
      file_path = os.path.join(output_folder+'/'+file_name+'.csv')
      df = pd.read_csv(file_path)
      df = df.drop(columns=['.geo', 'Shape_Area', 'Shape_Leng'], axis=1)
      df['Year'] = df['system:index'].str.slice(start=0, stop=4)
      df = df.drop('system:index', axis=1).rename(columns={'mean':name})
      break 

  return df

In [ ]:
new_df = spatial_reduction(image_collection=means_max_temp, 
                  shape_file = subset,
                  reducer = ee.Reducer.mean(),
                  scale=1000,
                  file_name = 'function_try',
                  format = 'CSV',
                  output_folder = 'climate_stats',
                  name = 'max_temp_mean')

{'state': 'READY', 'description': 'function_try', 'creation_timestamp_ms': 1653077709471, 'update_timestamp_ms': 1653077709471, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'ICAP2LDAVQ3GJGL3ZZQXFQL6', 'name': 'projects/earthengine-legacy/operations/ICAP2LDAVQ3GJGL3ZZQXFQL6'}
{'state': 'RUNNING', 'description': 'function_try', 'creation_timestamp_ms': 1653077709471, 'update_timestamp_ms': 1653077717786, 'start_timestamp_ms': 1653077717750, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': 'ICAP2LDAVQ3GJGL3ZZQXFQL6', 'name': 'projects/earthengine-legacy/operations/ICAP2LDAVQ3GJGL3ZZQXFQL6'}


In [ ]:
## ------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION ##
## ------------------------------------------- ##

years = np.arange(1986,2022)
sum_pr = []
for i in years:
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-10-01'), (str((i+1))+'-9-30')) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index':str(i)}) #select the bands to analyze
  sum_pr.append(prcp) #calculate the mean across all pixels

sum_precip = ee.ImageCollection(sum_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(sum_precip, precip_vis, 'prcp')
Map

In [ ]:
## ------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION ##
## ------------------------------------------- ##

years = np.arange(1986,2022)
irrig_pr = []
for i in years:
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-03-15'), (str((i+1))+'-10-15')) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index':str(i)}) #select the bands to analyze
  irrig_pr.append(prcp) #calculate the mean across all pixels

irrig_precip = ee.ImageCollection(irrig_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(sum_precip, precip_vis, 'prcp')
Map

In [ ]:
new_df = spatial_reduction(image_collection=sum_precip, 
                  shape_file = subset,
                  reducer = ee.Reducer.mean(),
                  scale=1000,
                  file_name = 'precip_sums_try',
                  format = 'CSV',
                  output_folder = 'climate_stats',
                  name = 'precip_sum_mean')

{'state': 'READY', 'description': 'precip_sums_try', 'creation_timestamp_ms': 1653069937204, 'update_timestamp_ms': 1653069937204, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': '2JSBXBHBHK4V24J5NFP7U3HZ', 'name': 'projects/earthengine-legacy/operations/2JSBXBHBHK4V24J5NFP7U3HZ'}


In [ ]:
## ----------------------------- ##
## 4. IMPORT MONTHLY PRECIP DATA ##
## ----------------------------- ##

years = np.arange(1987,2021)
months = np.arange(3,11)
sum_month_pr = []

for i in years:
  for m in months:
    if (m == 3) or (m == 5) or (m == 7) or (m == 8) or (m==10):
      daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-'+str(m)+'-01'), (str(i)+'-'+str(m)+'-31')) #get image collection
      prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set('system:index', (str(i)+'-'+str(m))) #select the bands to analyze
      sum_month_pr.append(prcp) #calculate the mean across all pixels
    else: 
      daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-'+str(m)+'-01'), (str(i)+'-'+str(m)+'-30')) #get image collection
      prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set('system:index', (str(i)+'-'+str(m))) #select the bands to analyze
      sum_month_pr.append(prcp) #calculate the mean across all pixels

sum_month_precip = ee.ImageCollection(sum_month_pr) #convert list of image to image collection for zonal stats command


In [ ]:
## IMPORT MONTHLY ET DATA ##

et_irrig = []

for y in years:
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-03-01'), str(y)+'-10-31')
  et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(y)}) # sum and convert to meters 
  et_irrig.append(et)

et_irrig = ee.ImageCollection(et_irrig)

et_cut = []

for y in years:
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-04-01'), str(y)+'-9-30')
  et_march = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-03-01'), str(y)+'-03-31')
  et_march = et_march.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': (str(y)+'-03')})
  et_oct = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-10-01'), str(y)+'-10-31')
  et_oct = et_oct.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': (str(y)+'-10')})
  et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(y)}) # sum and convert to meters 
  et_cut.append(et)
  et_cut.append(et_march)
  et_cut.append(et_oct)

et_cut = ee.ImageCollection(et_cut)
  

In [ ]:

## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('climate_stats/maxtemp_stats.csv')
emap.zonal_statistics(means_max_temp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/precip_stats.csv')
emap.zonal_statistics(sum_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/irrig_precip_stats.csv')
emap.zonal_statistics(irrig_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/month_precip_stats.csv')
emap.zonal_statistics(sum_month_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/et.csv')
emap.zonal_statistics(et_irrig, subset, out_stats, statistics_type='MEAN', scale=30)

out_stats = os.path.join('climate_stats/et_cut.csv')
emap.zonal_statistics(et_cut, subset, out_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/maxtemp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/precip_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/irrig_precip_stats.csv
Computing statistics ...


Exception: ignored

In [6]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

years = np.arange(1987,2021)
precip = pd.read_csv('climate_stats/precip_stats.csv')
max_temp = pd.read_csv('climate_stats/maxtemp_stats.csv')
irrig_pre = pd.read_csv('climate_stats/irrig_precip_stats.csv')
et_irrig = pd.read_csv('climate_stats/et.csv')
et_cut_ir = pd.read_csv('climate_stats/et_cut.csv')

names = et_cut_ir['WaterRight']

cut = pd.DataFrame(et_cut_ir['WaterRight'])
for i in years:
  fill = str(i)
  cut[fill] = et_cut_ir[et_cut_ir.columns[et_cut_ir.columns == (str(i)+'_et')]]


for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['Precip_mm'] = precip.iloc[i,1:35].values
  df['Max_temp'] = max_temp.iloc[i,0:34].values
  df['et_whole'] = et_irrig.iloc[i,0:34].values
  df['et_cut'] = cut.iloc[i,1:35].values
  out_path = os.path.join('climate_stats/final/'+names[i]+'_climate.csv')
  display(df)
  df.to_csv(out_path)


,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Andrews,172.645691,30.86208,0.863127,0.738347
1,1988,Andrews,285.013735,32.672246,0.797447,0.722437
2,1989,Andrews,187.442381,31.477965,0.620083,0.586516
3,1990,Andrews,189.587751,31.65148,0.857457,0.774692
4,1991,Andrews,205.834636,31.404859,0.825613,0.771539
5,1992,Andrews,379.021946,31.580158,0.933268,0.840096
6,1993,Andrews,167.494446,27.220322,0.770403,0.686474
7,1994,Andrews,356.114704,32.895048,0.93805,0.827853
8,1995,Andrews,281.831136,29.885366,0.879878,0.756042
9,1996,Andrews,310.88774,32.171572,0.867531,0.735208


,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Atwell,176.950001,30.643847,1.08367,0.911433
1,1988,Atwell,301.96,32.456043,0.841864,0.757452
2,1989,Atwell,197.72,31.427252,1.149675,1.038284
3,1990,Atwell,206.65,31.591759,1.019514,0.889469
4,1991,Atwell,205.44,31.283627,1.039115,0.965961
5,1992,Atwell,380.730001,31.402967,0.913124,0.777396
6,1993,Atwell,175.269998,27.067034,1.023808,0.880189
7,1994,Atwell,380.099999,32.727253,1.150544,0.968748
8,1995,Atwell,317.37,29.81967,1.130122,0.961381
9,1996,Atwell,347.05,32.045494,1.241228,1.04236


,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Ballentyne,219.031531,30.360606,0.95555,0.852817
1,1988,Ballentyne,319.063866,32.28887,0.947439,0.890583
2,1989,Ballentyne,281.652087,31.421353,0.99994,0.895612
3,1990,Ballentyne,228.655105,31.32197,1.036027,0.930805
4,1991,Ballentyne,231.787944,31.040294,0.929388,0.873681
5,1992,Ballentyne,440.985037,30.939893,1.001564,0.8964
6,1993,Ballentyne,202.808041,26.50369,0.821259,0.746552
7,1994,Ballentyne,419.955856,32.416002,0.988905,0.874973
8,1995,Ballentyne,411.074099,29.514785,1.010695,0.880809
9,1996,Ballentyne,447.289639,31.738827,0.978944,0.820216


,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Barber,222.219816,30.356307,0.978828,0.887926
1,1988,Barber,320.60426,32.299286,0.858432,0.818039
2,1989,Barber,282.403703,31.426821,0.963956,0.854366
3,1990,Barber,232.945925,31.313177,1.033523,0.888915
4,1991,Barber,231.752779,31.049939,0.982739,0.942506
5,1992,Barber,434.870557,30.973395,0.987542,0.907564
6,1993,Barber,199.522964,26.512885,0.871844,0.782186
7,1994,Barber,417.600554,32.423955,1.216272,1.069858
8,1995,Barber,408.100741,29.491823,1.157084,1.000258
9,1996,Barber,448.035187,31.698609,1.177401,0.991533


,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Bates,172.394356,30.864997,0.893849,0.767621
1,1988,Bates,282.014267,32.678832,0.834061,0.754882
2,1989,Bates,185.195956,31.475972,0.616532,0.581457
3,1990,Bates,190.462978,31.652389,0.757337,0.685497
4,1991,Bates,203.911201,31.405197,0.844553,0.790001
5,1992,Bates,376.011689,31.585589,0.778359,0.690025
6,1993,Bates,166.350755,27.225711,0.723006,0.654684
7,1994,Bates,353.467689,32.907589,0.850773,0.740638
8,1995,Bates,276.857955,29.889731,0.893119,0.775197
9,1996,Bates,308.325555,32.169616,0.968795,0.821933


...

,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Thomas Aiken,224.413523,30.39601,1.281437,1.165441
1,1988,Thomas Aiken,322.733493,32.341792,1.231446,1.1726
2,1989,Thomas Aiken,290.302149,31.449237,1.152546,1.038744
3,1990,Thomas Aiken,231.810478,31.334535,1.167625,1.032624
4,1991,Thomas Aiken,233.157941,31.06535,1.161444,1.089479
5,1992,Thomas Aiken,440.250715,30.9501,1.247795,1.129904
6,1993,Thomas Aiken,200.888358,26.509247,0.96095,0.868714
7,1994,Thomas Aiken,419.421015,32.450312,1.223352,1.103753
8,1995,Thomas Aiken,412.100179,29.523238,1.168163,1.022119
9,1996,Thomas Aiken,453.540507,31.740559,1.17654,1.01871


,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Thurman Mill,236.868436,30.42842,0.962384,0.849821
1,1988,Thurman Mill,337.857188,32.371372,0.958364,0.889409
2,1989,Thurman Mill,299.237535,31.495637,1.027026,0.914166
3,1990,Thurman Mill,246.131247,31.330642,0.985458,0.883931
4,1991,Thurman Mill,240.744062,31.085228,0.895263,0.833745
5,1992,Thurman Mill,445.594802,30.969245,0.952012,0.854526
6,1993,Thurman Mill,204.156694,26.489669,0.774912,0.701805
7,1994,Thurman Mill,427.412764,32.489024,0.924518,0.811323
8,1995,Thurman Mill,421.067098,29.469209,0.999573,0.838167
9,1996,Thurman Mill,473.246828,31.686617,0.96451,0.818414


,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Upper Centerpoint,173.743512,30.704086,0.796354,0.673644
1,1988,Upper Centerpoint,295.658643,32.515047,0.821662,0.766561
2,1989,Upper Centerpoint,191.021707,31.447489,0.652329,0.620031
3,1990,Upper Centerpoint,203.74581,31.627582,0.864589,0.771872
4,1991,Upper Centerpoint,203.197286,31.307861,0.80153,0.75201
5,1992,Upper Centerpoint,374.724869,31.459345,0.871085,0.776875
6,1993,Upper Centerpoint,171.78023,27.124166,0.885817,0.767011
7,1994,Upper Centerpoint,374.838337,32.777292,0.996362,0.858466
8,1995,Upper Centerpoint,307.976653,29.857409,0.905068,0.779173
9,1996,Upper Centerpoint,330.343982,32.088011,0.921228,0.797511


,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Wagner,175.07,30.594397,0.817358,0.696458
1,1988,Wagner,275.43,32.422855,0.701982,0.658753
2,1989,Wagner,202.56,31.434505,0.760976,0.675133
3,1990,Wagner,218.810002,31.586264,0.766738,0.685217
4,1991,Wagner,207.829999,31.269781,0.591925,0.549909
5,1992,Wagner,391.86,31.381428,0.67937,0.589289
6,1993,Wagner,187.380001,27.047142,0.619052,0.567715
7,1994,Wagner,384.890002,32.701099,0.928057,0.806028
8,1995,Wagner,330.710001,29.815495,0.815578,0.663881
9,1996,Wagner,353.009999,32.028572,0.808994,0.66852


,Year,DIV_NAME,Precip_mm,Max_temp,et_whole,et_cut
0,1987,Warm Springs,224.624849,30.39189,1.103298,0.997727
1,1988,Warm Springs,322.606164,32.340458,1.068796,1.00729
2,1989,Warm Springs,288.785602,31.445234,1.154084,1.042377
3,1990,Warm Springs,234.922849,31.334732,1.097761,0.978567
4,1991,Warm Springs,232.945261,31.07256,1.081211,1.020322
5,1992,Warm Springs,437.746837,30.968933,1.132115,1.043714
6,1993,Warm Springs,199.709617,26.524097,0.791732,0.728642
7,1994,Warm Springs,419.19504,32.455408,1.05504,0.94972
8,1995,Warm Springs,410.091795,29.514194,1.148201,0.995529
9,1996,Warm Springs,453.629767,31.724796,1.170578,1.0


In [ ]:
## --------------- ## 
## COMPARE ET DATA ##
## --------------- ## 

years = np.arange(1987,2021)
et_whole = pd.read_csv('climate_stats/et.csv').drop(columns = ['Shape_Area', 'system:index', 'Shape_Leng'], axis=1)
et_cut = pd.read_csv('climate_stats/et_cut.csv')


names = et_whole['WaterRight']
et = []

march = pd.DataFrame(et_cut['WaterRight'])
october = pd.DataFrame(et_cut['WaterRight'])
cut = pd.DataFrame(et_cut['WaterRight'])

for i in years:
  fill = str(i)
  march[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'-03_et')]]
  october[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'-10_et')]]
  cut[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'_et')]]

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['et_whole'] = et_whole.iloc[i,0:34].values
  df['et_cut'] = cut.iloc[i,1:35].values
  df['march'] = march.iloc[i,1:35].values
  df['october'] = october.iloc[i,1:35].values
  et.append(df)

et = pd.concat(et)

et['diff'] = et['et_whole'] - et['et_cut']

avg = et.groupby('DIV_NAME')['et_whole', 'et_cut', 'diff', 'march', 'october'].mean()
avg.to_csv('climate_stats/et_avgs.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [ ]:
## ------------------------- ##
## COPY ET DATA to Cloud API ##
## ------------------------- ##

# folder from where to copy
src_folder = "projects/earthengine-legacy/assets/users/dgketchum/ssebop/boise"
# folder where to copy
dest_folder = "projects/ee-bridgetbittmann/assets/ssebop"

# get all assets in the folder
assets = ee.data.listAssets({'parent': src_folder})

# loop through assets and copy them one by one to the new destination
for asset in assets['assets']:
    # construct destination path
    new_asset = dest_folder + '/' + asset['id'].split('/')[-1]
    # copy to destination
    ee.data.copyAsset(asset['id'], new_asset, True)

In [ ]:
## ------------------------------------------- ##
## COPY ET DATA from Cloud API to Earth Engine ##
## ------------------------------------------- ##

# folder from where to copy
src_folder = "projects/ee-bridgetbittmann/assets/ssebop"
# folder where to copy
dest_folder = "projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise"

# get all assets in the folder
assets = ee.data.listAssets({'parent': src_folder})

# loop through assets and copy them one by one to the new destination
for asset in assets['assets']:
    # construct destination path
    new_asset = dest_folder + '/' + asset['id'].split('/')[-1]
    # copy to destination
    ee.data.copyAsset(asset['id'], new_asset, True)